In [ ]:
import pandas as pd
file = "data/4import_Database_Austria_March2018.xlsx"
df = pd.read_excel(file, "MONUMENT PROTECTION")[4:]

In [ ]:
df

In [ ]:
for i, row in df.iterrows():
    if row['*SiteID']:
        temp_monument, _ = MonumentProtection.objects.get_or_create(
            identifier=row['*Monument Protection ID']
        )
        if row['Public'] == "yes":
            temp_monument.public = True
        # hook to site
        temp_site, _ = Site.objects.get_or_create(
            identifier=row['*SiteID']
        )
        temp_monument.site_id = temp_site
        
        if row['Threats']:
            try:
                for x in row['Threats'].split(','):
                    temp_method,_ = SkosConcept.objects.get_or_create(
                        pref_label=x
                    )
                    temp_monument.threats.add(temp_method)
                    temp_monument.save()
            except AttributeError:
                pass
                
        
        temp_monument.heritage_status = row['Heritage Status']
        temp_monument.save()
    